In [19]:
import sys

def dijkstra(adj_matrix, start_city, end_city):
    num_cities = len(adj_matrix)
    distances = [sys.maxsize] * num_cities
    visited = [False] * num_cities
    previous = [None] * num_cities

    distances[start_city] = 0

    for _ in range(num_cities):
        min_dist = sys.maxsize
        current_city = None

        for city in range(num_cities):
            if not visited[city] and distances[city] < min_dist:
                min_dist = distances[city]
                current_city = city

        visited[current_city] = True

        if current_city == end_city:
            break

        for neighbor in range(num_cities):
            if (
                not visited[neighbor]
                and adj_matrix[current_city][neighbor] != 0
                and distances[current_city] + adj_matrix[current_city][neighbor] < distances[neighbor]
            ):
                distances[neighbor] = distances[current_city] + adj_matrix[current_city][neighbor]
                previous[neighbor] = current_city

    if distances[end_city] == sys.maxsize:
        return "No path found."

    path = []
    current_city = end_city
    while current_city is not None:
        path.append(current_city)
        current_city = previous[current_city]

    path.reverse()
    return path, distances[end_city]

In [20]:
import random
import numpy as np
import matplotlib.pyplot as plt

num_villes = 50
NumTruck = 3

def generate_weighted_adjacency_matrix(n):
    # Générer une matrice d'adjacence vide
    adjacency_matrix = [[0] * n for _ in range(n)]

    # Générer des liens aléatoires
    for i in range(n):
        for j in range(i + 1, n):
            if random.random() < 0.6:  # Probabilité de lien entre deux villes
                weight = random.randint(1, 1000)  # Poids du lien entre deux villes
                adjacency_matrix[i][j] = weight 
                adjacency_matrix[j][i] = weight
                

    return adjacency_matrix

adj_matrix = generate_weighted_adjacency_matrix(num_villes)

validSolutions = []

# initialise list of visited cities
visitedCities = [[] for i in range(NumTruck)]

# Determine initial cities (can't have 2 times the same)
truckJourneys = []

numValidSol = 10

for n in range(numValidSol):
    for i in range(NumTruck):
        initCity = random.randint(1,num_villes)
        while initCity in truckJourneys:
            initCity = random.randint(1,num_villes)
        truckJourneys.append([initCity])
        visitedCities[i].append(initCity)
    print("Initial cities : ", truckJourneys)

    truckJourneysFinished = 0

    while truckJourneysFinished != NumTruck:
        truckJourneysFinished = 0
        for j in range(NumTruck):
            if len(visitedCities[j]) != num_villes:
                curTruck = j
                curCity = truckJourneys[curTruck][-1]
                neighborsCurCity = adj_matrix[curCity]
                closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > 0))
                CityFound = False
                minValue = 0
                while CityFound == False:
                    if max(neighborsCurCity) == minValue:
                        closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > 0))
                        if truckJourneys[curTruck][-2] == closerCity:
                            while neighborsCurCity[closerCity] == 0 or closerCity == truckJourneys[curTruck][-2]:
                                closerCity = neighborsCurCity.index(random.choice(neighborsCurCity))
                            truckJourneys[curTruck].append(closerCity)
                            CityFound = True
                        else:
                            closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > 0))
                            truckJourneys[curTruck].append(closerCity)
                            CityFound = True
                    else:
                        closerCity = neighborsCurCity.index(min(i for i in neighborsCurCity if i > minValue))
                        if closerCity in visitedCities[j]:
                            minValue = neighborsCurCity[closerCity]
                        else:
                            truckJourneys[curTruck].append(closerCity)
                            visitedCities[j].append(closerCity)
                            CityFound = True
        for k in range(NumTruck):
            if len(visitedCities[k]) == num_villes:
                truckJourneysFinished += 1
    
    for j in range(len(truckJourneys)):
        result = dijkstra(adj_matrix, truckJourneys[j][-1], truckJourneys[j][0])
        path, distance = result
        truckJourneys[j] = truckJourneys[j] + path
    
    for j in range(len(visitedCities)):
        print("Truck Journey ", j, " : ", truckJourneys[j])
    
    for j in range(len(visitedCities)):
        print("With Duplicate", len(truckJourneys[j]))

    for j in range(len(visitedCities)):
        print("Without Duplicate", len(set(truckJourneys[j])))
        
    validSolutions.append(truckJourneys)
    visitedCities = [[] for i in range(NumTruck)]
    truckJourneys = []

print(validSolutions)
print(len(validSolutions))

Initial cities :  [[29], [29], [33]]
Truck Journey  0  :  [29, 45, 38, 42, 19, 37, 13, 1, 0, 12, 2, 3, 43, 23, 46, 6, 36, 35, 14, 24, 4, 26, 30, 27, 16, 47, 32, 17, 22, 31, 28, 40, 20, 25, 41, 39, 7, 33, 34, 9, 21, 5, 49, 10, 11, 44, 48, 18, 15, 8, 8, 34, 3, 33, 29]
Truck Journey  1  :  [29, 45, 38, 42, 19, 37, 13, 1, 0, 12, 2, 3, 43, 23, 46, 6, 36, 35, 14, 24, 4, 26, 30, 27, 16, 47, 32, 17, 22, 31, 28, 40, 20, 25, 41, 39, 7, 33, 34, 9, 21, 5, 49, 10, 11, 44, 48, 18, 15, 8, 8, 34, 3, 33, 29]
Truck Journey  2  :  [33, 29, 45, 38, 42, 19, 37, 13, 1, 0, 12, 2, 3, 43, 23, 46, 6, 36, 35, 14, 24, 4, 26, 30, 27, 16, 47, 32, 17, 22, 31, 28, 40, 20, 25, 41, 39, 7, 48, 9, 34, 8, 10, 49, 5, 21, 18, 11, 44, 23, 15, 15, 22, 17, 32, 12, 2, 3, 33]
With Duplicate 55
With Duplicate 55
With Duplicate 59
Without Duplicate 50
Without Duplicate 50
Without Duplicate 50
Initial cities :  [[42], [35], [27]]
Truck Journey  0  :  [42, 19, 37, 13, 1, 0, 12, 2, 3, 43, 23, 46, 6, 36, 35, 14, 24, 4, 26, 30, 27, 16,